In [2]:
import pandas as pd

# Notebook for adding New compounds to table

### 1. Run this cell to load the table

In [3]:
# load table from parquet file
compound_table = pd.read_parquet('compound_table.parquet')

# sort values by unique idea so last has highest id
compound_table.sort_values('id', ascending=True, inplace=True)

# electrochem_table = pd.read_parquet('electrochem_table.parquet')
# photochem_table = pd.read_parquet('photochem_table.parquet')
# solubility_table = pd.read_parquet('solubility_table.parquet')


### 2. Enter the details of the compound you want to add below, then run the cell

In [32]:
compound_details = {
                  'name': ' (dimethylaminomethyl)ferrocene (FcN)', 
                  'formula': 'C13H17FeN', 
                  'molecular_weight': 243, 
                  'CAS': '1271-86-9', 
                  'source_dois': 'DOI:10.1021/jacs.6b10984, https://www.sigmaaldrich.com/GB/en/product/aldrich/119342', 
                  'recyclable': 1, 
                  'type': 'electron donor', 
                  'compound_family': 'ferrocene', 
                  'used_in_photocat': 0, 
                  'used_in_rfbs': 1, 
                  'used_as_hcarrier': 0}

### 3. Run this cell to add your compound to the table
This code includes a duplication check.

In [33]:
# create unique compound id and add compound

# if the table is empty add the compound with id 1
if compound_table.empty:
    compound_details['id'] = int(1)
    compound_table = pd.concat([compound_table, pd.DataFrame([compound_details,])])
    
    
else:
    # check if compounds already exist with the same combination of name, formula and CAS
    check_mask = compound_table.isin([compound_details['name'], compound_details['formula'], compound_details['CAS']])
    check_mask = check_mask.loc[(check_mask.name==True) & (check_mask.formula==True), :]
    
    # if the CAS is a blank string ignore CAS in your checks
    if compound_details['CAS'] == '':
        check_mask.CAS = False
    
    
    # if no compounds with matching name, formula and CAS are found add one to the last id number and add the compound with that id
    if check_mask.any(axis=1).empty:
        # this assumes table always ordered ascending to make addition faster
        compound_details['id'] = int(compound_table.id.iat[-1] + 1)
        compound_table = pd.concat([compound_table, pd.DataFrame([compound_details,])])
        
    else:
        # if matches are found print them with a warning message and don't add the compound yet
        print(f"The following compounds appear to match the one you are trying to add {check_table.name.to_list()}.")
        print("To edit an existing entry please use the compound editor notebook. Otherwise please change the name or formula of your compound to add it.")
        display(compound_table.loc[check_mask.any(axis=1), :])
        print("Your compound has not yet been added")
    

#### (Execute next cell to preview last 5 entries)

In [34]:
# code to preview last five entries
if compound_table.shape[0] > 6:
    display(compound_table.tail(5))
else:
    display(compound_table)


,id,name,formula,molecular_weight,CAS,source_dois,recyclable,type,compound_family,used_in_photocat,used_in_rfbs,used_as_hcarrier
0,14,"2-(1,3-dimethyl-2H-benzimidazol-2-yl)phenol (H...",C15H16N2O,293.0,3652-93-5,"doi:10.1016/j.tet.2006.03.061, https://pubchem...",0,electron donor,benzimidazole,1,0,0
0,15,(ferrocenylmethyl)trimethylammonium chloride (...,C14H20ClFeN,293.0,83617-79-2,"DOI:10.1021/jacs.6b10984, https://pubchem.ncbi...",1,electron donor,ferrocene,0,1,0
0,16,"N1-ferrocenylmethyl-N1,N1,N2,N2,N2-pentamethyl...",C19H32Br2FeN2,502.0,,"DOI:10.1021/jacs.6b10984,,",1,electron donor,ferrocene,0,1,0
0,17,methyl viologen (MV),C12H14Cl2N2,257.0,1910-42-5,"DOI:10.1021/jacs.6b10984, https://pubchem.ncbi...",1,electron donor,ferrocene,1,1,0
0,18,(dimethylaminomethyl)ferrocene (FcN),C13H17FeN,243.0,1271-86-9,"DOI:10.1021/jacs.6b10984, https://www.sigmaald...",1,electron donor,ferrocene,0,1,0


### 4. Run the next cell to save your changes to the table 

In [35]:
compound_table.to_parquet('compound_table.parquet', engine='pyarrow', compression=None)